In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from FeatureExtractor import *

In [3]:
stack = 'MD594'

first, last = section_range_lookup[stack]
dms = dict([(sc, DataManager(stack=stack, section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(first, last+1)])

In [4]:
labels = ['Pn', '12N', 'Gr', "7N", '7n', "5N", 'VLL', 'LVe', 'SuVe']

In [26]:
label_polygon = pickle.load(open(os.environ['REPO_DIR']+'/database/label_polygon.pkl', 'r'))

In [27]:
patch_dir = '/home/yuncong/CSHL_data_patches/'

In [28]:
h, w = dms[first].image_height, dms[first].image_width

patch_size = 224
half_size = patch_size/2
stride = 56

In [29]:
dense_xs, dense_ys = np.meshgrid(np.arange(0, h), np.arange(0, w), indexing='ij')

In [32]:
for sec in range(first, last+1):
#     labelmap = np.zeros((grid_ys.size,), np.uint8)

    sys.stderr.write('%d\n'% sec)

    labelmap = np.zeros((h,w), np.uint8)

    for label_ind, label in enumerate(labels):
        if sec not in label_polygon[label]:
            continue
        
        sys.stderr.write('%s\n'% label)
            
        vertices = label_polygon[label][sec]
        polygon_path = Path(vertices)

        xmin, ymin  = np.min(vertices, axis=0).astype(np.int)
        xmax, ymax  = np.max(vertices, axis=0).astype(np.int)
    #     width = xmax - xmin + 1
    #     height = ymax - ymin + 1

        lm_xs, lm_ys = np.meshgrid(np.arange(xmin, xmax+1), np.arange(ymin, ymax+1))
        lm_xs_flat = lm_xs.flat
        lm_ys_flat = lm_ys.flat
        indices_inside = np.where(polygon_path.contains_points(np.c_[lm_xs_flat, lm_ys_flat]))[0]
        labelmap[lm_ys_flat[indices_inside], lm_xs_flat[indices_inside]] = label_ind + 1
        
#         indices_inside = np.where(polygon_path.contains_points(np.c_[xs.flat, ys.flat]))[0]
#         labelmap[indices_inside] = label_ind + 1
    
#     np.save(patch_dir+'/dense_labelmap/labelmap_%d.npy' % sec, labelmap)
    np.save(patch_dir+'/dense_labelmap2/labelmap_%d.npy' % sec, labelmap)

49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
VLL
74
VLL
75
VLL
76
VLL
77
5N
VLL
78
7N
7n
5N
VLL
79
7n
5N
VLL
LVe
SuVe
80
7N
7n
5N
VLL
LVe
SuVe
81
7N
7n
5N
VLL
LVe
SuVe
82
7N
7n
5N
VLL
LVe
SuVe
83
7N
7n
5N
VLL
LVe
SuVe
84
7N
7n
5N
VLL
LVe
SuVe
85
7N
7n
5N
VLL
LVe
SuVe
86
7N
7n
5N
VLL
LVe
SuVe
87
7N
7n
VLL
LVe
88
7N
7n
VLL
LVe
89
Pn
7N
7n
LVe
90
Pn
7N
7n
91
Pn
7N
7n
92
Pn
7N
7n
93
Pn
7N
7n
94
Pn
7N
7n
95
Pn
7N
7n
96
Pn
7N
7n
97
Pn
Gr
7N
7n
98
Pn
Gr
7N
7n
99
Pn
Gr
7N
7n
100
Pn
Gr
7n
101
Pn
Gr
7n
102
Pn
Gr
7n
103
Pn
Gr
7n
104
Pn
Gr
105
Pn
12N
Gr
106
Pn
12N
Gr
107
Pn
12N
Gr
108
Pn
12N
Gr
109
Pn
12N
Gr
110
Pn
12N
Gr
111
Pn
12N
Gr
112
113
Pn
12N
Gr
114
Pn
12N
115
Pn
12N
Gr
116
Pn
12N
Gr
117
12N
Gr
118
12N
Gr
119
12N
Gr
120
12N
Gr
121
12N
Gr
122
12N
Gr
123
12N
Gr
124
12N
Gr
125
Pn
12N
Gr
126
Pn
Gr
127
Pn
Gr
128
Pn
Gr
129
Pn
Gr
130
Pn
Gr
131
Pn
Gr
132
Pn
Gr
133
Pn
Gr
134
Pn
135
Pn
136
Pn
137
Pn
7n
138
Pn
7n
139
Pn
7N
7n
140
Pn
7N
7n
141
Pn
7N
7n
142
Pn
7N
7n
143
Pn

In [40]:
np.max(img_as_ubyte(viz))

255

In [35]:
viz = label2rgb(l)

In [39]:
imwrite('viz.jpg', img_as_ubyte(viz))

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


True

In [ ]:
# xs, ys = np.meshgrid(np.arange(half_size, h-half_size), np.arange(half_size, w-half_size), indexing='ij')

grid_ys, grid_xs = np.meshgrid(np.arange(half_size, h-half_size, stride), 
                               np.arange(half_size, w-half_size, stride), 
                               indexing='ij')

In [ ]:
for sec in range(first, last+1):
    labelmap = np.zeros((grid_ys.size,), np.uint8)

    for label_ind, label in enumerate(labels):
        if sec not in label_polygon[label]:
            continue
        vertices = label_polygon[label][sec]
        polygon_path = Path(vertices)

    #     xmin, ymin  = np.min(vertices, axis=0).astype(np.int)
    #     xmax, ymax  = np.max(vertices, axis=0).astype(np.int)
    #     width = xmax - xmin + 1
    #     height = ymax - ymin + 1

    #     xs, ys = np.meshgrid(np.arange(xmin, xmax+1), np.arange(ymin, ymax+1))
        indices_inside = np.where(polygon_path.contains_points(np.c_[grid_xs.flat, grid_ys.flat]))[0]
        labelmap[indices_inside] = label_ind + 1
        
    np.save(patch_dir+'/labelmap/labelmap_%d.npy' % sec, labelmap)

In [ ]:
labelmap = {}
neighbor_labels = defaultdict(set)
landmark_positions = defaultdict(dict)
bg_positions = defaultdict(dict)

for sec in range(first, last+1):
    labelmap[sec] = np.load(patch_dir+'/labelmap/labelmap_%d.npy' % sec).reshape(grid_xs.shape)
    
    for label_ind, label in enumerate(labels):
        
        if label_ind + 1 in labelmap[sec]:
            ys, xs = np.where(labelmap[sec] == label_ind + 1)
            
            landmark_positions[sec][label] = np.c_[xs, ys]
            
            ymin = np.min(ys)
            ymax = np.max(ys)
            xmin = np.min(xs)
            xmax = np.max(xs)
            
            a = set(np.unique(labelmap[sec][max(0, ymin-500/stride):ymax+500/stride, 
                                            max(0, xmin-500/stride):xmax+500/stride])) - {0, label_ind+1}
            neighbor_labels[label_ind] |= set([i-1 for i in a])
            
            surround_ys, surround_xs = np.where(labelmap[sec][max(0, ymin-500/stride):ymax+500/stride, 
                                                              max(0, xmin-500/stride):xmax+500/stride] == 0)
            bg_positions[sec][label] = np.c_[surround_xs + max(0, xmin-500/stride),
                                             surround_ys + max(0, ymin-500/stride)]
            
#             print sec, label, set([labels[i-1] for i in a])
            
landmark_positions.default_factory = None
neighbor_labels.default_factory = None
bg_positions.default_factory = None

In [ ]:
pickle.dump(landmark_positions, open(patch_dir+'landmark_positions.pkl', 'wb'))
pickle.dump(neighbor_labels, open(patch_dir+'neighbor_labels.pkl', 'wb'))
pickle.dump(bg_positions, open(patch_dir+'bg_positions.pkl', 'wb'))

In [ ]:
# z = np.zeros_like(grid_xs)
# for label, vertices in bg_positions[106].iteritems():
#     for x, y in vertices:
#         z[y,x] = labels.index(label) + 1
# plt.matshow(z);

In [ ]:
# nearby landmark classes
[(labels[label_ind], [labels[n] for n in neighbors]) for label_ind, neighbors in neighbor_labels.iteritems()]

In [ ]:
# for sec in range(first, last+1):
for sec in range(73, last+1):
    
    sys.stderr.write('%d\n'%sec)
    
    if sec in bg_positions:

        dms[sec]._load_image(['rgb'])
        
        for label_ind, label in enumerate(labels):
            if label not in bg_positions[sec]: continue

            patches = []

            inds = bg_positions[sec][label]
            xs = grid_xs[inds[:,1], inds[:,0]]
            ys = grid_ys[inds[:,1], inds[:,0]]
            for i in range(xs.size):
                x = xs[i]
                y = ys[i]
                patch = dms[sec].image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
                patches.append(patch)

            patches_arr = np.array([img_as_ubyte(p) for p in patches], np.uint8)
            np.save(patch_dir+'/patches/patchesBg_%s_%d.npy'%(label, sec), patches_arr)
        
    if sec in landmark_positions:

        dms[sec]._load_image(['rgb'])
        
        for label_ind, label in enumerate(labels):
            if label not in landmark_positions[sec]: continue

            patches = []

            inds = landmark_positions[sec][label]
            xs = grid_xs[inds[:,1], inds[:,0]]
            ys = grid_ys[inds[:,1], inds[:,0]]
            for i in range(xs.size):
                x = xs[i]
                y = ys[i]
                patch = dms[sec].image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
                patches.append(patch)

            patches_arr = np.array([img_as_ubyte(p) for p in patches], np.uint8)
            np.save(patch_dir+'/patches/patches_%s_%d.npy'%(label, sec), patches_arr)
                
    if hasattr(dms[sec], 'image_rgb'):        
        del dms[sec].image_rgb